In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 7774.28it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4589.53it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2330799.66it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8370.40it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4534.20it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 90990.29it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8001.52it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4812.91it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 93976.55it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 8573.24it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4706.40it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 92631.28it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.DRMMTKSModel()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['top_k'] = 20
model.params['embedding_input_dim'] = preprocessor.context['vocab_size'] + 1
model.params['embedding_output_dim'] = 100
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 5
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DRMMTKSModel.
Parameter "embedding_trainable" set to True.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1667400     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_1 (Dot)               

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = model.EvaluateOnCall(model, x=pred_x, y=pred_y, batch_size=len(pred_x), valid_steps=1)

In [10]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=20)
len(train_generator)

102

In [11]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 4s 40ms/step - loss: 0.9350


Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.411158 - normalized_discounted_cumulative_gain@5(0):0.480680 - mean_average_precision(0):0.443565


Epoch 2/30
102/102 [==============================] - 3s 31ms/step - loss: 0.8480


Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.410229 - normalized_discounted_cumulative_gain@5(0):0.493076 - mean_average_precision(0):0.446971


Epoch 3/30
102/102 [==============================] - 3s 30ms/step - loss: 0.8387


...

Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.603593 - normalized_discounted_cumulative_gain@5(0):0.654189 - mean_average_precision(0):0.611697


Epoch 29/30
102/102 [==============================] - 3s 31ms/step - loss: 0.1565


Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.593457 - normalized_discounted_cumulative_gain@5(0):0.655041 - mean_average_precision(0):0.614009


Epoch 30/30
102/102 [==============================] - 3s 31ms/step - loss: 0.1463


Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.594946 - normalized_discounted_cumulative_gain@5(0):0.655295 - mean_average_precision(0):0.609338


In [12]:
import bokeh
from bokeh.io import output_notebook, push_notebook
from bokeh.layouts import column
from bokeh.models.glyphs import Line
from bokeh.models.tools import HoverTool
from bokeh.plotting import figure, show, Figure

In [13]:
charts = {
    metric: figure(
        title=str(metric),
        sizing_mode='scale_width',
        width=800, height=400
    ) for metric in history.history.keys()
}
for metric, sub_chart in charts.items():
    x = history.epoch
    y = history.history[metric]
    lines = sub_chart.line(x, y, line_width=2, alpha=0.5)
output_notebook()
show(charts[list(charts.keys())[1]])
show(charts[list(charts.keys())[2]])
show(charts[list(charts.keys())[3]])
#show(column(*charts.values()))

Loading BokehJS ...

In [14]:
from keras.utils import plot_model
plot_model(model.backend, to_file='model.png')

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.